1.Download the dataset

2.Import required library

In [26]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from keras.utils import to_categorical
from keras.models import load_model

3.Read Dataset and do preprocessing


In [27]:
df = pd.read_csv('/content/drive/MyDrive/spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [28]:
df.drop(['Unnamed: 2','Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True) #dropping unwanted columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [29]:
#Count of Spam and Ham values
df.groupby(['v1']).size()

v1
ham     4825
spam     747
dtype: int64

In [30]:
#Label Encoding target column
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [31]:
# Test and train spilit 
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [32]:
# Tokenisation function
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

4.Create Models and 5. Add Layers(LSTM,Dense,(Hidden Layers),Output)

In [33]:
#creating LSTM model
inputs = Input(name='InputLayer',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FullyConnectedLayer1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='OutputLayer')(layer)
layer = Activation('sigmoid')(layer)

6.Compile the model

In [34]:
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputLayer (InputLayer)     [(None, 150)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 150, 50)           50000     
                                                                 
 lstm_1 (LSTM)               (None, 64)                29440     
                                                                 
 FullyConnectedLayer1 (Dense  (None, 256)              16640     
 )                                                               
                                                                 
 activation_2 (Activation)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           

7.Fit the Model

In [35]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2)

Epoch 1/10
30/30 [==============================] - 10s 265ms/step - loss: 0.3404 - accuracy: 0.8640 - val_loss: 0.1503 - val_accuracy: 0.9652
Epoch 2/10
30/30 [==============================] - 8s 251ms/step - loss: 0.0916 - accuracy: 0.9768 - val_loss: 0.0646 - val_accuracy: 0.9842
Epoch 3/10
30/30 [==============================] - 7s 248ms/step - loss: 0.0475 - accuracy: 0.9865 - val_loss: 0.0567 - val_accuracy: 0.9842
Epoch 4/10
30/30 [==============================] - 7s 246ms/step - loss: 0.0349 - accuracy: 0.9910 - val_loss: 0.0515 - val_accuracy: 0.9873
Epoch 5/10
30/30 [==============================] - 7s 248ms/step - loss: 0.0288 - accuracy: 0.9921 - val_loss: 0.0518 - val_accuracy: 0.9852
Epoch 6/10
30/30 [==============================] - 7s 250ms/step - loss: 0.0234 - accuracy: 0.9929 - val_loss: 0.0622 - val_accuracy: 0.9863
Epoch 7/10
30/30 [==============================] - 7s 246ms/step - loss: 0.0170 - accuracy: 0.9952 - val_loss: 0.0697 - val_accuracy: 0.9852
Epoch

8.Save the Model

In [36]:
model.save("model_1")

9.Test the Model

In [40]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [21]:
y_pred =model.predict(test_sequences_matrix)
print(y_pred[25:40].round(3))

148/148 [==============================] - 3s 20ms/step
[[0.   ]
 [0.   ]
 [1.   ]
 [0.005]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [1.   ]
 [0.   ]
 [0.   ]
 [0.001]
 [0.   ]
 [0.   ]]


In [42]:
accuracy = model.evaluate(test_sequences_matrix,Y_test)
print('Accuracy: {:0.3f}'.format(accuracy[1]))

27/27 [==============================] - 1s 19ms/step - loss: 0.0583 - accuracy: 0.9844
Accuracy: 0.984


In [22]:
print(Y_test[25:40])

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]]
